# Introduction
This data set includes top 50 Spotify songs in 2019.It has 13 columns.

      * track_name    50 non-null     object
      * artist_name   50 non-null     object
      * Genre         50 non-null     object

Three of them is object(string).

      * b_p_m         50 non-null     int64 
      * Energy        50 non-null     int64 
      * Danceability  50 non-null     int64 
      * loudness_db   50 non-null     int64 
      * Liveness      50 non-null     int64 
      * valence       50 non-null     int64 
      * length        50 non-null     int64 
      * Acousticness  50 non-null     int64 
      * Speechiness.  50 non-null     int64 
      * Popularity    50 non-null     int64 
 
The columns which are above, their data types is integer.

I'm going to do EDA(Explotary Data Analysis) and Visualization of this dataset.

<font color = blue>
Content : 

1. [Load and Check Data](#1)   
1. [Variable Description](#2)        
1. [Basic Data Analysis](#3) 
1. [Outlier Detection](#4)     
1. [Visualization with Seaborn](#5)     
    
    

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import plotly as py
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id="1"></a><br>
## Load and Check Data
In this part I load data into sData variable and I changed some of the column names for usability. I removed the Unnamed: 0 column also.

In [ ]:
def load_data():
    sData = pd.read_csv('/kaggle/input/top50spotify2019/top50.csv', encoding="windows-1252")
    sData = sData.rename(columns = {"Speechiness." : "Speechiness","Unnamed: 0" : "id","Track.Name" : "track_name", "Artist.Name" : "artist_name", "Beats.Per.Minute" : "b_p_m", "Loudness..dB.." : "loudness_db", "Valence." : "valence", "Length." : "length","Acousticness.." : "Acousticness" })
    sData = sData.drop(["id"], axis = 1)
    return sData
sData = load_data()
sData.head()


In [ ]:
sData.info()

There are 13 columns and 3 of them is object. Other columns are integer.

In [ ]:
sData.track_name.value_counts()  #track name is unique value

In [ ]:
sData.track_name.isnull()

In [ ]:
sData.Genre.value_counts()

In [ ]:
sData.artist_name.value_counts()

In [ ]:
sData.corr()

<a id = '2'></a></br>
## Variable Description
1. track_name 	
1. artist_name 	
1. Genre 	
1. b_p_m 	
1. Energy 	
1. Danceability 	
1. loudness_db 	
1. Liveness 	
1. valence 	
1. length 	
1. Acousticness 	
1. Speechiness. 	
1. Popularity


<font color = red>
Categorical Analysis : 
    
    * track_name 	
    * artist_name 	
    * Genre 	
    
<font color = red>
Numerical Analysis :
    
    * b_p_m
    * Energy
    * Danceability
    * loudness_db
    * Liveness
    * valence
    * length
    * Acousticness
    * Speechiness.
    * Popularity

## Categorical Analysis

Plotting bar plots according to columns which have data type object.

In [ ]:
def bar_plot(attribute):
    variable = sData[attribute]
    count = variable.value_counts()
    
    plt.figure(figsize=(10,8))
    plt.bar(count.index, count)
    plt.xlabel(attribute)
    plt.ylabel("Frequency")
    plt.xticks(count.index, count.index.values, rotation = 90)
    plt.title(attribute + "Frequency")
    plt.show()

In [ ]:
categories = ["track_name", "artist_name", "Genre" ]
for each in categories:
    bar_plot(each)

## Numerical Analysis
Plotting histogram according to columns which have data type integer.

In [ ]:
#change data type int to float to use it effectively.
def int_to_float(variable):
    sData[variable] = sData[variable].astype(float)
    
num = ["b_p_m", "Energy", "Danceability", "loudness_db", "Liveness", "valence", "length", "Acousticness", "Speechiness", "Popularity"]
for n in num:
    int_to_float(n)

In [ ]:
sData.info()

In [ ]:
def plot_histogram(attribute):
    
    plt.figure(figsize=(10,8))
    plt.hist(sData[attribute], bins = 50 )
    plt.xlabel(attribute)
    plt.ylabel("Counts")
    plt.xticks(rotation = 90)
    plt.show()

In [ ]:
numericals = ["b_p_m", "Energy", "Danceability", "loudness_db", "Liveness", "valence", "length", "Acousticness", "Speechiness", "Popularity"]

for each in numericals:
    plot_histogram(each)

<a id ="3"></a>
## Basic Data Analysis
In this part, I made a simple analysis with using 'groupby' method.

In [ ]:
sData[["Energy", "Danceability"]].groupby("Danceability", as_index = False).mean().sort_values(by = "Energy", ascending = False)

In [ ]:
sData[["b_p_m", "Energy", "Danceability"]].groupby(["b_p_m", "Energy"], as_index = False).min().sort_values(by = "b_p_m", ascending = False)

<a id = "4"></a>
## Outlier Detection
We calculate the outliers only from numeric values.

In [ ]:
sData.boxplot("Energy")
plt.show()

In [ ]:
def outliers(data, variables):
    
    outliers_list = []
    
    for each in variables:
        
        Q1 = np.percentile(data[each], 25)
        
        Q3 = np.percentile(data[each], 75)
        
        IQR = Q3 - Q1
        
        outlier_step = 1.5 * IQR
        
        outlier_list_col = data[(data[each] < Q1 - outlier_step) | (data[each] > Q3 + outlier_step)].index
        
        outliers_list.extend(outlier_list_col)
        
    outliers_list = Counter(outliers_list)
    print(outliers_list)
    multiple_outliers = list(i for i, v in outliers_list.items() if v > 2)
    return multiple_outliers

In [ ]:
sData.loc[outliers(sData, ["b_p_m", "Energy", "Danceability", "loudness_db", "Liveness", "valence", "length", "Acousticness", "Speechiness", "Popularity"])]

There is no outlier rows in data set.

In [ ]:
sData.columns[sData.isnull().any()]

<a id="5"></a>
## Visualization with Seaborn

In [ ]:
genres = sData.Genre.value_counts()

plt.figure(figsize= (15,10))
sns.barplot(x = genres.index, y = genres.values, color = "cyan", alpha = 0.5, label = "Genre - Frequency")
plt.xticks(rotation = 90)
plt.show()

In [ ]:
artists = sData["artist_name"].value_counts()

plt.figure(figsize= (15,10))
sns.barplot(x = artists.index, y = artists.values, color = "cyan", alpha = 0.5, label = "Genre - Frequency")
plt.xticks(rotation = 90)
plt.show()

In [ ]:
sData.artist_name.unique()

In [ ]:
#sData[["artist_name","Genre"]].groupby("Genre", as_index = False).max().sort_values(by = "Genre", ascending = False)

In [ ]:

f, ax = plt.subplots(figsize = (15,10))
sns.pointplot(x = sData.track_name , y = sData.Popularity, color = "green", alpha = 0.5)
plt.title("Track Name / Popularity")
plt.xticks(rotation = 90)
plt.show()

In [ ]:
sns.lmplot(x = "b_p_m", y = "loudness_db", data = sData)
plt.show()

In [ ]:
sData.info()

In [ ]:
plt.figure(figsize = (10,10))
sns.boxplot(x = "Genre", y = "Energy", hue = "length", data = sData)
plt.plot()

With using pair plot, we can see the all attributes

In [ ]:
sns.pairplot(sData)
plt.show()

In [ ]:
plt.figure(figsize = (10,10))
sns.swarmplot(x = "Popularity", y = "Liveness", hue = "Genre", data = sData)
plt.show()

In [ ]:
a = sData.Genre.value_counts()
labels = a.index
colors = ["red","blue","cyan","green"]
explode = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
print(len(explode))
sizes = a.values

plt.figure(figsize = (10,10))
plt.pie(sizes, explode = explode, labels = labels, colors = colors, autopct = '%1.1f%%')
plt.show()


In [ ]:
artis = sData.artist_name.value_counts()

labels = artis.index
colors = ["red","blue","cyan","green"]
explode = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
size = artis.values

plt.figure(figsize = (10,10))
plt.pie(size, explode = explode, colors = colors, labels = labels, autopct = '%1.1f%%')
plt.show()

In [ ]:
f,ax = plt.subplots(figsize = (10,7))
sns.heatmap(sData.corr(), annot = True, linewidth = .8, fmt = '.1f', ax = ax, linecolor = 'cyan')
plt.show()

In [ ]:
sns.countplot(sData.Energy)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
#popularity scores of genres
genres = sData.Genre.unique()
popularity_scores = []

for g in genres:
    var = sData[sData.Genre == g]
    popularity = sum(var.Popularity)
    popularity_scores.append(popularity)

plt.figure(figsize = (10,10))
sns.barplot(x = genres, y = popularity_scores, palette = sns.cubehelix_palette(len(genres)))
plt.xticks(rotation = 90)
plt.xlabel("Genres")
plt.ylabel("Total popularity scores")
plt.title("Genre/Popularity")
plt.show()


In [ ]:
#artist_name = artis.index[::-1]
artist_name = artis.index
artist_popularity = []

for a in artist_name:
    artist = sData[sData.artist_name == a]
    popularity = sum(artist.Popularity)
    artist_popularity.append(popularity)

plt.figure(figsize = (10,10))
sns.barplot(x = artist_name, y = artist_popularity, palette = sns.cubehelix_palette(len(artist_name)))
plt.xticks(rotation = 90)
plt.title("Artist/Popularity")
plt.show()

## Visualization with Plotly

In [ ]:
trace = go.Scatter(
    x = sData.track_name,
    y = sData.Energy,
    mode = 'markers+lines',
    marker = dict(color = "cyan"),
    text = sData.Energy,
)
trace2 = go.Scatter(
    x = sData.track_name,
    y = sData.b_p_m,
    mode = 'markers',
    marker = dict(color = "red"),
    text = sData.b_p_m,
)

data = [trace, trace2]
layout = dict(title = "Track Name - Energy - BPM", xaxis = dict(title = 'Track Name', ticklen = len(sData.track_name), zeroline = False))
fig = dict(data = data, layout = layout)
iplot(fig)

In [ ]:
sData.info()

In [ ]:
sData.head()

In [ ]:
trace = go.Bar(
    x = sData.Genre,
    y = sData.Danceability,
    name = "Dancebility",
    marker = dict(
        color = "cyan"
    )
)

trace2 = go.Bar(
    x = sData.Genre,
    y = sData.Energy,
    name = "Energy",
    marker = dict(
        color = "red"
    )
)

data = [trace, trace2]
layout = go.Layout(barmode = 'group')
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [ ]:
trace = go.Bar(
    x = sData.Genre,
    y = sData.Danceability,
    name = "Dancebility",
    marker = dict(
        color = "cyan"
    )
)

trace2 = go.Bar(
    x = sData.Genre,
    y = sData.Energy,
    name = "Energy",
    marker = dict(
        color = "red"
    ),
)

data = [trace, trace2]
layout = go.Layout(barmode = 'overlay')
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [ ]:
trace = go.Scatter(
    x = artist_name,
    y = artist_popularity,
    name = "Popularity",
    mode = 'markers+lines',
    marker = dict( color = "cyan"),
    text = artist_name,
)

data = [trace]
layout = dict( title = "Artist's Total Popularity Scores")
fig = dict(data = data, layout = layout)
iplot(fig)

In [ ]:
trace2 = go.Scatter(
    x = artist_name,
    y = artis.values,
    name = "Track Number",
    mode = 'markers+lines',
    marker = dict( color = "red"),
    text = artis.values,
)

data = [ trace2]
layout = dict( title = "Artist's Popular Track Numbers")
fig = dict(data = data, layout = layout)
iplot(fig)

In [ ]:
trace = go.Box(
    y = sData.Energy,
    name = 'Energy',
    marker = dict(color = "red")
)
data = [trace]
iplot(data)